In [1]:
import sys
import os
import pandas as pd
import numpy as np

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

In [60]:
import torch
from torch import nn
from torch import functional as F

batch_size = 32
block_size = 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
DATA_PATH = os.path.join(PROJECT_ROOT, "data")
PATH_RATINGS = os.path.join(DATA_PATH, "raw_data", "kaggle_second_sem", "books_rating.csv")
PATH_BOOKS = os.path.join(DATA_PATH, "raw_data", "kaggle_second_sem", "books_data.csv")
PATH_EMBDS = os.path.join(DATA_PATH, "embeddings", "expanded_embds_ss.npy")

df_books = pd.read_csv(PATH_BOOKS)
df_ratings = pd.read_csv(PATH_RATINGS)
book_embds = np.load(PATH_EMBDS, allow_pickle=True)

# Book embds has shape like
#(Title, (count, 1), (review/score, 1) (author, 1), (categories, 1), (publisher, 1), (description_embd, 384))

In [4]:
ratings_seqs = df_ratings.groupby("User_id")["Title"].apply(lambda x: list(set(x.tolist()))).loc[lambda x: x.str.len() > 5].reset_index()
train = ratings_seqs["Title"].iloc[:30000]
test = ratings_seqs["Title"].iloc[30000:]

In [5]:
torch.randint(2, (3, 3))

# torch.randint(len(train[1636]) - block_size, (1,))
torch.randint(len(train[20551]) - block_size, (1,))

tensor([11])

In [95]:
book_embds_dict = {row[0]: np.array(row[1:], dtype=np.float32) for row in book_embds}

def encode(book_title):
    return book_embds_dict[book_title]
def encode_seq(seq):
    return [book_embds_dict[title] for title in seq]

In [94]:
def clear_nan(sample):
    return sample[np.array([all(isinstance(j, str) for j in seq) for seq in sample])]

train = clear_nan(train)
test = clear_nan(test)

train.shape, test.shape

((29962,), (16459,))

In [96]:
def get_batch(split):
    data = train.reset_index(drop=True) if split == 'train' else test.reset_index(drop=True)
    ix = torch.randint(len(train), (batch_size,))

    x = []
    y = []
    for i in ix:
        i = int(i)       
        j = torch.randint(len(data[i]) - block_size-1, (1,))
        x.append(encode_seq(data[i][j:j + block_size]))
        y.append(encode(data[i][j + block_size + 1]))

    x = torch.tensor(x)
    y = torch.tensor(y)
    x, y = x.to(device), y.to(device)
    return x, y

In [88]:
X, y = get_batch('train')
X.shape, y.shape

(torch.Size([32, 4, 389]), torch.Size([32, 389]))

In [9]:
def get_max_index(col_idx):
    return int(max(
        embd[1 + col_idx]
        for embd in book_embds_dict.values()
    ))

count_size = get_max_index(0) + 100000
review_size = get_max_index(1) + 100000
author_size = get_max_index(2) + 1000000
cat_size = get_max_index(3) + 100000
publisher_size = get_max_index(4) + 100000


In [12]:
count_size, review_size, author_size, cat_size, publisher_size

(100005, 227278, 1010883, 116016, 100001)

In [10]:
count_embd_size = max(train)

sm_embd = 20
desc_embd = 384
n_embd = sm_embd * 5 + desc_embd

In [ ]:
484 / 4 

121.0

In [109]:
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        self.attn = nn.MultiheadAttention(n_embd, 11, batch_first=True)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, 16 * n_embd),
            nn.ReLU(),
            nn.Linear(16 * n_embd, n_embd)
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x), self.ln1(x), self.ln1(x))[0]
        x = x + self.mlp(self.ln2(x))
        return x

class Optimus(nn.Module):
    def __init__(self):
        super().__init__()
        self.count_embds = nn.Embedding(count_size, sm_embd, device=device)
        self.review_embds = nn.Embedding(review_size, sm_embd, device=device)
        self.author_embds = nn.Embedding(author_size, sm_embd, device=device)
        self.cat_embds = nn.Embedding(cat_size, sm_embd, device=device)
        self.publisher_embds = nn.Embedding(publisher_size, sm_embd, device=device)
        self.pos_embds = nn.Embedding(block_size, n_embd, device=device)
        self.blocks = nn.Sequential(*[Block() for _ in range(20)])
        self.ln_head = nn.Linear(n_embd, 5 + desc_embd, device=device)  
        
    def forward(self, x):
        B, T, C = x.shape

        x = torch.cat([
            self.count_embds(x[:, :, 0].long()),
            self.review_embds(x[:, :, 1].long()),
            self.author_embds(x[:, :, 2].long()),
            self.cat_embds(x[:, :, 3].long()),
            self.publisher_embds(x[:, :, 4].long()),
            x[:, :, 5:]
        ], dim=-1)

        pos_embd = self.pos_embds(torch.arange(T, device=x.device)).unsqueeze(0)
        x = x + pos_embd

        x = self.blocks(x)

        x = x[:, -1, :]
        x = self.ln_head(x)

        return x


In [110]:
model = Optimus().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

In [ ]:
loss_fn = nn.CosineEmbeddingLoss()


loss_avg = 0
for step in range(10000):
    xb, yb = get_batch('train')  
    
    out = model(xb) 
    
    loss = loss_fn(out, yb, torch.ones(32))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    loss_avg += loss.item()
    if step % 40 == 0:
        print(f"Step {step}: Loss = {loss_avg / 40:.4f}")
        loss_avg = 0


In [ ]:
model(get_batch('train')[0]).shape

In [ ]:
get_batch('train')[0][:, :, 2].long()